In [2]:
!pip3 install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.1 MB/s eta 0:00:00


In [3]:
import gymnasium
from gymnasium import spaces, utils
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from operator import add
from math import sqrt

# Creating the Random Maze Environment using the Gymnasium.Env class

In [9]:
class RandomMazeEnv(gymnasium.Env):

    metadata = {"render_modes" : ["human", "rgb_array"], "render_fps": 4}
    # Define constants for clearer code
    LEFT = 0
    DOWN = 3
    RIGHT = 2
    UP = 1

    def __init__(self, render = None, seed = None):
        super(RandomMazeEnv, self).__init__()

        p = 0.8
        self.grid_size = 3 * 4
        # Number of discrete actions, we have four: left, down, right, up
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Discrete(12)
        self.P = {
            0: {
                0: [
                    [p + (1-p)/2, 0, -0.04, False],
                    [(1-p)/2, 4, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 0, -0.04, False],
                    [p, 4, -0.04, False],
                    [(1-p)/2, 1, -0.04, False]
                ],
                2: [
                    [(1-p)/2, 4, -0.04, False],
                    [p, 1, -0.04, False],
                    [(1-p)/2, 0, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 1, -0.04, False],
                    [p + (1-p)/2, 0, -0.04, False],
                ]
            },
            1: {
                0: [
                    [(1-p), 1, -0.04, False],
                    [p, 0, -0.04, False],
                ],
                3: [
                    [(1-p)/2, 0, -0.04, False],
                    [p, 1, -0.04, False],
                    [(1-p)/2, 2, -0.04, False]
                ],
                2: [
                    [(1-p), 1, -0.04, False],
                    [p, 2, -0.04, False],
                ],
                1: [
                    [(1-p)/2, 2, -0.04, False],
                    [p, 1, -0.04, False],
                    [(1-p)/2, 0, -0.04, False]
                ]
            },
            2: {
                0: [
                    [(1-p)/2, 2, -0.04, False],
                    [p, 1, -0.04, False],
                    [(1-p)/2, 6, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 1, -0.04, False],
                    [p, 6, -0.04, False],
                    [(1-p)/2, 3, 1.0, True]
                ],
                2: [
                    [(1-p)/2, 6, -0.04, False],
                    [p, 3, 1.0, True],
                    [(1-p)/2, 2, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 3, 1.0, True],
                    [p, 2, -0.04, False],
                    [(1-p)/2, 1, -0.04, False]
                ]
            },
            3: {
                0: [
                    [1.0, 3, 0.0, True]
                ],
                3: [
                    [1.0, 3, 0.0, True]
                ],
                2: [
                    [1.0, 3, 0.0, True]
                ],
                1: [
                    [1.0, 3, 0.0, True]
                ]
            },
            4: {
                0: [
                    [(1-p)/2, 0, -0.04, False],
                    [p, 4, -0.04, False],
                    [(1-p)/2, 8, -0.04, False]
                ],
                3: [
                    [(1-p), 4, -0.04, False],
                    [p, 8, -0.04, False],
                ],
                2: [
                    [(1-p)/2, 8, -0.04, False],
                    [p, 4, -0.04, False],
                    [(1-p)/2, 0, -0.04, False]
                ],
                1: [
                    [(1-p), 4, -0.04, False],
                    [p, 0, -0.04, False],
                ]
            },
            5: {
                0: [
                    [1.0, 5, 0.0, True]
                ],
                3: [
                    [1.0, 5, 0.0, True]
                ],
                2: [
                    [1.0, 5, 0.0, True]
                ],
                1: [
                    [1.0, 5, 0.0, True]
                ]
            },
            6: {
                0: [
                    [(1-p)/2, 2, -0.04, False],
                    [p, 6, -0.04, False],
                    [(1-p)/2, 10, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 6, -0.04, False],
                    [p, 10, -0.04, False],
                    [(1-p)/2, 7, -1.0, True]
                ],
                2: [
                    [(1-p)/2, 10, -0.04, False],
                    [p, 7, -1.0, True],
                    [(1-p)/2, 2, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 7, -1.0, True],
                    [p, 2, -0.04, False],
                    [(1-p)/2, 6, -0.04, False]
                ]
            },
            7: {
                0: [
                    [1.0, 7, 0.0, True]
                ],
                3: [
                    [1.0, 7, 0.0, True]
                ],
                2: [
                    [1.0, 7, 0.0, True]
                ],
                1: [
                    [1.0, 7, 0.0, True]
                ]
            },
            8: {
                0: [
                    [(1-p)/2, 4, -0.04, False],
                    [p + (1-p)/2, 8, -0.04, False]
                ],
                3: [
                    [p + (1-p)/2, 8, -0.04, False],
                    [(1-p)/2, 9, -0.04, False]
                ],
                2: [
                    [(1-p)/2, 8, -0.04, False],
                    [p, 9, -0.04, False],
                    [(1-p)/2, 4, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 9, -0.04, False],
                    [p, 4, -0.04, False],
                    [(1-p)/2, 8, -0.04, False]
                ]
            },
            9: {
                0: [
                    [(1-p), 9, -0.04, False],
                    [p, 8, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 8, -0.04, False],
                    [p, 9, -0.04, False],
                    [(1-p)/2, 10, -0.04, False]
                ],
                2: [
                    [(1-p), 9, -0.04, False],
                    [p, 10, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 10, -0.04, False],
                    [p, 9, -0.04, False],
                    [(1-p)/2, 8, -0.04, False]
                ]
            },
            10: {
                0: [
                    [(1-p)/2, 6, -0.04, False],
                    [p, 9, -0.04, False],
                    [(1-p)/2, 10, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 9, -0.04, False],
                    [p, 10, -0.04, False],
                    [(1-p)/2, 11, -0.04, False]
                ],
                2: [
                    [(1-p)/2, 10, -0.04, False],
                    [p, 11, -0.04, False],
                    [(1-p)/2, 6, -0.04, False]
                ],
                1: [
                    [(1-p)/2, 11, -0.04, False],
                    [p, 6, -0.04, False],
                    [(1-p)/2, 9, -0.04, False]
                ]
            },
            11: {
                0: [
                    [(1-p)/2, 7, -1.0, True],
                    [p, 10, -0.04, False],
                    [(1-p)/2, 11, -0.04, False]
                ],
                3: [
                    [(1-p)/2, 10, -0.04, False],
                    [p + (1-p)/2, 11, -0.04, False]
                ],
                2: [
                    [p + (1-p)/2, 11, -0.04, False],
                    [(1-p)/2, 7, -1.0, True]
                ],
                1: [
                    [(1-p)/2, 11, -0.04, False],
                    [p, 7, -1.0, True],
                    [(1-p)/2, 10, -0.04, False]
                ]
            }
        }
        self.seed(seed)
        self.start_state = 8
        self.state = self.start_state
        self.gamma = 0.99 # discount factor

    def seed(self, seed=None):
        self.np_random, seed = gymnasium.utils.seeding.np_random(seed)
        return [seed]

    def step(self, action):
        transitions = self.P[self.state][action]
        i = self.np_random.choice(len(transitions), p=[t[0] for t in transitions])
        prob, next_state, reward, done = transitions[i]
        self.state = next_state
        return next_state, reward, done, {}

        # moving to the next state
        self.state = next_state
        return next_state, reward, done, {}

    def reset(self, seed = None):
        self.seed(seed)
        # Reset the state of the environment to an initial state
        self.state = self.start_state
        return self.state

    def close(self):
         pass

# Implementing the `policyEvaluation()`, `policyImprovement()` and `policyIteration()` algorithm for the MDP of the Random Maze Experiment

In [10]:
# Implementing the policy evaluation condition
def policyEvaluation(policy, P, gamma, theta):
    values_old = np.zeros(len(P.keys()))
    while 1:
        values_new = np.zeros(len(P.keys()))
        # iterating over all states and actions
        for state in P:
            for action in P[state]:
                factor = 0
                if action == policy[state]:
                    factor = 1
                temp = 0
                for prob, next_state, reward, _ in P[state][action]:
                    temp += prob * (reward + gamma * values_old[next_state])
                values_new[state] += factor * temp

        # checking termination condition
        diff_values = [abs(v1 - v2) for v1, v2 in zip(values_old, values_new)]
        if max(diff_values) < theta:
            break
        values_old = values_new

    return values_new

# implementing the policy iteration
def policyImprovement(values, P, gamma):
  q_values = np.zeros((len(P.keys()), 4))
  new_policy = np.zeros(len(P.keys()), dtype=int)
  for state in P:
    for action in P[state]:
      for prob, next_state, reward, _ in P[state][action]:
          q_values[state][action] += prob * (reward + gamma * values[next_state])

    new_policy[state] = np.argmax(q_values[state])

  # Returning the updated policy
  return new_policy

# Implementing the policy iteration
def policyIteration(P, gamma, theta):
  values_final = np.zeros(len(P.keys()))
  policy_adverserial = [2,2,2,3,1,2,3,1,2,3,3,2]

  iterations = 0
  while True:
    values = policyEvaluation(policy_adverserial, P, gamma, theta)
    policy_new = policyImprovement(values, P, gamma)
    iterations += 1
    # checking the termination condition
    if np.array_equal(policy_adverserial, policy_new):
      values_final = values
      break

    policy_adverserial = policy_new

  return policy_new, values_final, iterations

In [11]:
global_seed = 123
gamma = 0.99
theta = 10e-10
env = RandomMazeEnv(seed = global_seed)

print(len(env.P.keys()))

optimal_policy, optimal_values, iterations = policyIteration(env.P, gamma, theta)

print("Optimal Value Function:", optimal_values)
print("Optimal Policy:", optimal_policy)
print("Iterations:", iterations)

12
Optimal Value Function: [0.82442985 0.89286374 0.95464233 0.         0.76427487 0.
 0.68820946 0.         0.69763948 0.63906542 0.60613373 0.38186228]
Optimal Policy: [2 2 2 0 1 0 1 0 1 0 1 0]
Iterations: 3


# Implementing the `valueIteration()` algorithm

In [19]:
# Implemeting the value iteration algorithm
def valueIteration(P, gamma, theta):

    values = np.zeros(len(P.keys()))  # Initialize value function
    pi = dict()  # Initialize policy
    iterations = 0

    while True:
        iterations += 1
        delta = 0
        for state in P.keys():
            values_state = values[state]
            Q_state_action = np.zeros(4)
            for action in P[state].keys():
                for prob, next_state, reward, _ in P[state][action]:
                    Q_state_action[action] += prob * (reward + gamma * values[next_state])
            values[state] = np.max(Q_state_action)
            delta = max(delta, abs(values_state - values[state]))

        # Break out of the loop when delta is less than theta
        if delta < theta:
            break

    # Derive policy from value function
    for state in P.keys():
        Q_state_action = np.zeros(4)
        for action in P[state].keys():
            for prob, next_state, reward, _ in P[state][action]:
                Q_state_action[action] += prob * (reward + gamma * values[next_state])
        pi[state] = np.argmax(Q_state_action)  # Choose action that maximizes the action-value function

    return pi, values, iterations


In [20]:
global_seed = 123
gamma = 0.99
theta = 10e-10
env = RandomMazeEnv(seed = global_seed)

print(len(env.P.keys()))

optimal_policy, optimal_values, iterations = valueIteration(env.P, gamma, theta)

print("Optimal Value Function:", optimal_values)
print("Optimal Policy:", optimal_policy)
print("Iterations:", iterations)

12
Optimal Value Function: [0.82442985 0.89286374 0.95464233 0.         0.76427487 0.
 0.68820946 0.         0.69763948 0.63906542 0.60613373 0.38186228]
Optimal Policy: {0: 2, 1: 2, 2: 2, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0}
Iterations: 25
